In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import numpy as np
from collections import OrderedDict

from libs.commpy_mod import SISOFlatChannel

from files_01_detection.qam_crazy import crazy_channel_propagate
from files_01_detection.const_mod import generate_symbols, Model
from files_01_detection.const_analyzer import plot_decision_boundary, theoretical_error, ser, plot_confusion_matrix, plot_symbols
 
%matplotlib inline

2023-01-09 21:42:16.512629: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 21:42:16.714543: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-09 21:42:16.714565: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-09 21:42:16.746317: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-09 21:42:17.348137: W tensorflow/stream_executor/platform/de

In [2]:
Mod = 'QAM'
channel_type = 'rayleigh' # 'awgn' or 'crazy' or 'rayleigh'
M            = 4      # PSK modulation
total_num_symbols  = 30000    # Number of transmitted symbols to be used for training and test
train_fraction = 0.8 # Fraction of whole data to be used for training (the remaining is for testing)
SNR_dB       = 30      # Signal to noise ratio in dB     
code_rate    = 1       # Rate of the used code
Es           = 1       # Average symbol energy

In [3]:
symbs, indices, channel_output = Model(Mod, total_num_symbols, M, channel_type, Es, code_rate, SNR_dB)

In [4]:
symbs_ = symbs[:]

In [5]:
indices = list(OrderedDict.fromkeys(indices))

In [6]:
symbs = np.array(list(OrderedDict.fromkeys(symbs))).reshape(1,-1)

In [7]:
channel_output = channel_output.reshape(-1,1)

In [8]:
symb_min = []
for a in channel_output:
    symb_min.append(symbs[0, np.argmin(np.abs(a - symbs))])

In [11]:
d = dict(zip(symbs[0], indices))

In [18]:
for i in range(total_num_symbols):
    print(d[symb_min[i]])

1
1
3
3
2
2
0
0
1
0
3
2
0
0
1
2
3
2
2
3
1
0
2
3
2
2
0
3
0
2
1
1
1
2
2
2
0
1
1
0
2
3
0
2
3
1
2
3
2
0
0
1
1
2
1
2
3
2
0
0
0
3
1
0
2
3
3
2
3
3
3
2
2
0
3
0
1
1
2
0
0
3
2
2
2
1
0
2
3
0
0
0
2
3
1
0
0
0
1
0
3
3
0
2
2
3
0
1
3
3
1
0
3
1
2
3
3
2
1
0
3
0
3
2
0
0
2
0
0
0
3
2
2
0
1
1
1
3
3
1
1
1
1
3
1
0
3
3
3
2
1
2
1
2
2
3
0
3
1
2
2
1
1
1
1
2
3
1
2
1
2
3
2
0
0
1
1
3
0
0
0
1
1
1
3
3
2
2
0
0
0
1
1
1
0
3
3
1
2
3
1
2
0
2
3
3
1
1
1
1
2
3
0
1
3
2
1
0
0
2
3
2
1
2
0
0
3
2
0
2
1
3
0
1
0
1
3
2
0
1
3
2
3
2
1
1
1
3
1
0
2
3
3
2
3
0
0
0
1
1
3
2
3
3
3
1
1
1
0
1
2
1
0
2
0
3
0
3
2
2
2
2
3
2
1
1
0
0
3
1
0
3
1
3
2
1
3
1
2
0
0
1
3
1
3
1
1
1
1
0
1
3
0
2
1
1
2
1
2
1
0
1
2
1
0
1
1
2
3
0
3
3
1
2
2
2
3
0
1
0
2
1
3
0
3
0
0
3
2
0
2
1
1
3
1
2
2
1
2
3
0
3
0
1
0
2
0
1
2
1
1
0
1
0
1
0
0
3
3
3
3
0
1
0
1
1
2
2
0
1
1
0
2
3
1
2
0
0
2
1
1
3
1
0
0
0
0
3
3
1
2
3
2
2
0
1
2
2
3
2
1
3
2
2
0
3
2
1
1
3
1
1
2
0
2
3
0
0
3
1
3
1
1
0
2
3
0
0
3
2
2
1
1
2
1
1
0
1
2
3
3
1
0
0
1
1
3
2
3
2
3
0
0
1
2
0
2
0
1
1
1
1
1
3
2
3
1
1
0
0
2
0
3
1
0
1
1
2
2
0


In [19]:
for i in range(total_num_symbols):
    print(d[symbs_[i]])

1
1
3
3
2
2
0
0
1
0
3
2
0
0
1
2
3
2
2
3
1
0
2
3
2
2
0
3
0
2
1
1
1
2
2
2
0
1
1
0
2
3
0
2
3
1
2
3
2
0
0
1
1
2
1
2
3
2
0
0
0
3
1
0
2
3
3
2
3
3
3
2
2
0
3
0
1
1
2
0
0
3
2
2
2
1
0
2
3
0
0
0
2
3
1
0
0
0
1
0
3
3
0
2
2
3
0
1
3
3
1
0
3
1
2
3
3
2
1
0
3
0
3
2
0
0
2
0
0
0
3
2
2
0
1
1
1
3
3
1
1
1
1
3
1
0
3
3
3
2
1
2
1
2
2
3
0
3
1
2
2
1
1
1
1
2
3
1
2
1
2
3
2
0
0
1
1
3
0
0
0
1
1
1
3
3
2
2
0
0
0
1
1
1
0
3
3
1
2
3
1
2
0
2
3
3
1
1
1
1
2
3
0
1
3
2
1
0
0
2
3
2
1
2
0
0
3
2
0
2
1
3
0
1
0
1
3
2
0
1
3
2
3
2
1
1
1
3
1
0
2
3
3
2
3
0
0
0
1
1
3
2
3
3
3
1
1
1
0
1
2
1
0
2
0
3
0
3
2
2
2
2
3
2
1
1
0
0
3
1
0
3
1
3
2
1
3
1
2
0
0
1
3
1
3
1
1
1
1
0
1
3
0
2
1
1
2
1
2
1
0
1
2
1
0
1
1
2
3
0
3
3
1
2
2
2
3
0
1
0
2
1
3
0
3
0
3
3
2
0
2
1
1
3
1
2
2
1
2
3
0
3
0
1
0
2
0
1
2
1
1
0
1
0
1
0
0
3
3
3
3
0
1
0
1
1
2
2
0
1
1
0
2
3
1
2
0
0
2
1
1
3
1
0
0
0
0
3
3
1
2
3
2
2
0
1
2
2
3
2
1
3
2
2
0
3
2
1
1
3
1
1
2
0
2
3
0
0
3
1
3
1
1
0
2
3
0
0
3
2
2
1
1
2
1
1
0
1
2
3
3
1
0
0
1
1
3
2
3
2
3
0
0
1
2
0
2
0
1
1
1
1
1
3
2
3
1
1
0
0
2
0
3
1
0
1
1
2
2
0
